In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only, encoder_with_convs_and_symmetry_new

from tf_lab.nips.helper import average_per_class, zero_mean_half_sphere

from tf_lab.point_clouds.point_net_classifier import PointNetClassifier
from tf_lab.point_clouds.convenience import classify_pclouds

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
model_net = '40'
verbose = True
n_threads = 20
put_in_usphere = True
n_pc_points = 2048

In [5]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_points))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [6]:
pc, model_names, train_classes = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)
labels = np.array(classes_to_integers(int(model_net), train_classes)[1])

if put_in_usphere:
    pc = zero_mean_half_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, test_classes = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)
labels = np.array(classes_to_integers(int(model_net), test_classes)[1])

if put_in_usphere:
    pc = zero_mean_half_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [7]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'

In [8]:
experiment_name = 'sup_clf_2'

experiment_name = '_'.join([experiment_name, 'mnet', model_net,  'pts', str(n_pc_points)])

train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/sup_clf')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir);

In [ ]:
n_input = [n_pc_points, 3]
n_classes = int(model_net)

encoder = encoder_with_convs_and_symmetry_new
decoder = decoder_with_fc_only

decoder_args = {'layer_sizes': [256, 512, n_classes], 'b_norm': False}


encoder_args = {'n_filters': [64, 128, 256, 256, 512],
                'filter_sizes': [20, 5, 3, 3, 3],
                'pool_sizes': [2, 2, None, None, 2],
                'strides': [1, 1, 1, 1, 1],
                'b_norm': True,
                'symmetry': None
               }

conf = Conf(        
            n_input = n_input,
            n_output = [n_classes],
            training_epochs = 200,
            batch_size = 50,
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = 20,
            learning_rate = 0.0005,
            z_rotate = True, 
            encoder = encoder,
            decoder = decoder,
            encoder_args = encoder_args,
            decoder_args = decoder_args
            )


conf.one_hot = False
conf.experiment_name = experiment_name
conf.save(osp.join(conf.train_dir, 'configuration'))    
    
reset_tf_graph()
pnet_clf = PointNetClassifier(experiment_name, conf)

In [ ]:
c = conf
for _ in xrange(c.training_epochs):
    loss, duration = pnet_clf._single_epoch_train(train_data, c)
    epoch = int(pnet_clf.sess.run(pnet_clf.epoch.assign_add(tf.constant(1.0))))
    print 'TRAIN epoch: %d loss: %f' % (epoch, loss)
    if epoch % 2 == 0:
        avg_ac = classify_pclouds(pnet_clf, test_data.point_clouds, 100, test_data.labels)[1]
        print 'TEST epoch: %d accuracy: %f' % (epoch, avg_ac)

TRAIN epoch: 1 loss: 3.259875
TRAIN epoch: 2 loss: 1.357341
TEST epoch: 2 accuracy: 0.524929
TRAIN epoch: 3 loss: 1.173897
TRAIN epoch: 4 loss: 1.094668
TEST epoch: 4 accuracy: 0.642886
TRAIN epoch: 5 loss: 0.976022
TRAIN epoch: 6 loss: 0.882875
TEST epoch: 6 accuracy: 0.685853
TRAIN epoch: 7 loss: 0.834076
TRAIN epoch: 8 loss: 0.790343
TEST epoch: 8 accuracy: 0.672477
TRAIN epoch: 9 loss: 0.742379
TRAIN epoch: 10 loss: 0.720527
TEST epoch: 10 accuracy: 0.677341
TRAIN epoch: 11 loss: 0.632990
TRAIN epoch: 12 loss: 0.634466
TEST epoch: 12 accuracy: 0.674909
TRAIN epoch: 13 loss: 0.540367
TRAIN epoch: 14 loss: 0.551451
TEST epoch: 14 accuracy: 0.680584
TRAIN epoch: 15 loss: 0.500380
TRAIN epoch: 16 loss: 0.492762
TEST epoch: 16 accuracy: 0.657479
TRAIN epoch: 17 loss: 0.485130
TRAIN epoch: 18 loss: 0.448151
TEST epoch: 18 accuracy: 0.725578
TRAIN epoch: 19 loss: 0.392663
TRAIN epoch: 20 loss: 0.392889
TEST epoch: 20 accuracy: 0.672071
TRAIN epoch: 21 loss: 0.410037
TRAIN epoch: 22 loss: 

In [ ]:

# # if load_model:
# #     if load_epoch is None: # load last saved.
# #         saved_epochs = read_saved_epochs(conf.train_dir)
# #         load_epoch = saved_epochs[-1]
        
# #     ae.restore_model(conf.train_dir, load_epoch, verbose=True)